In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

#! pip install statsmodels

import statsmodels.api as sm

main_directory = Path.cwd().parent.as_posix()

In [12]:
df = pd.read_csv(f"{main_directory}/databases/zoopla_clean.csv")

In [13]:
def feature_eng(df):
    df = pd.get_dummies(df, columns = ['type'], drop_first = True, prefix = 'type')
    df = pd.get_dummies(df, columns = ['location'], drop_first = True, prefix = 'loc')
    df = sm.add_constant(df)
    return df

In [14]:
df

,type,price,station,beds,baths,receptions,post_code,location
0,terraced,160000.0,0.9,3,1,1,WN3,wigan
1,flat,235000.0,1.5,2,2,1,M20,manchester
2,flat,240000.0,1.5,2,2,1,M20,manchester
3,detached,499950.0,1.0,5,2,2,BL1,bolton
4,detached,289950.0,1.7,4,2,1,WN6,wigan
...,...,...,...,...,...,...,...,...
4695,flat,40000.0,0.3,1,1,0,BL1,lancashire
4696,flat,40000.0,0.3,1,1,0,BL1,lancashire
4697,flat,40000.0,0.3,1,1,0,BL1,lancashire
4698,flat,40000.0,0.3,1,1,0,BL1,lancashire


In [15]:
df = feature_eng(df)

feature_cols = list(df.columns)
feature_cols.remove('price')
feature_cols.remove('post_code') #Remove address -> not needed in the model
